In [77]:
%pip install pandas yfinance tqdm eikon

^C
Traceback (most recent call last):
  File "/Users/christopher/opt/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/christopher/opt/anaconda3/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/christopher/opt/anaconda3/lib/python3.9/site-packages/pip/__main__.py", line 31, in <module>
    sys.exit(_main())
  File "/Users/christopher/opt/anaconda3/lib/python3.9/site-packages/pip/_internal/cli/main.py", line 70, in main
    return command.main(cmd_args)
  File "/Users/christopher/opt/anaconda3/lib/python3.9/site-packages/pip/_internal/cli/base_command.py", line 101, in main
    return self._main(args)
  File "/Users/christopher/opt/anaconda3/lib/python3.9/site-packages/pip/_internal/cli/base_command.py", line 223, in _main
    self.handle_pip_version_check(options)
  File "/Users/christopher/opt/anaconda3/lib/python3.9/site-packages/pip/_internal/cli/req_command.py", 

In [130]:
import pandas as pd
import yfinance as yf
from tqdm import tqdm
import numpy as np
import eikon as ek
import requests

# Set Eikon API key
ek.set_app_key('e1df9a44dc6c4206a0599ca61d351c5a92689f71')

In [131]:
input_file = '/Users/christopher/Desktop/SPX_index leavers & joiners_29-Apr-2025.csv'

# Read the CSV file with joiners and leavers of S&P500
jl = pd.read_csv(input_file, delimiter=";", dtype=str)

In [132]:
date_fmt = "%d-%b-%Y"

# your jl DataFrame still has jl["Date"] as strings
# define a mapping from Danish → English abbrevs
month_map = {
    'jan': 'Jan', 'feb': 'Feb', 'mar': 'Mar', 'apr': 'Apr',
    'maj': 'May', 'jun': 'Jun', 'jul': 'Jul', 'aug': 'Aug',
    'sep': 'Sep', 'okt': 'Oct', 'nov': 'Nov', 'dec': 'Dec'
}

# use a regex replace so we only ever touch the middle (month) part
jl['Date'] = jl['Date'].str.replace(
    r'(\d{1,2})-([A-Za-z]+)-(\d{4})',
    lambda m: f"{m.group(1)}-{month_map.get(m.group(2).lower(), m.group(2))}-{m.group(3)}",
    regex=True
)

# now this will work:
jl['Date'] = pd.to_datetime(jl['Date'], format=date_fmt)

In [133]:
jl = (jl.rename(columns=str.strip)
         .loc[:, ["Status", "Issuer", "Code", "Date"]])

# chronological order within each RIC
jl = jl.sort_values(["Code", "Date"]).reset_index(drop=True)

In [134]:
jl

,Status,Issuer,Code,Date
0,Joiner,Agilent,A.N,2000-06-05
1,Joiner,Alcoa Corp,AA.N,2016-11-01
2,Leaver,Alcoa Corp,AA.N,2016-11-02
3,Joiner,Altaba,AABA.OQ^J19,1999-12-08
4,Leaver,Altaba,AABA.OQ^J19,2017-06-19
...,...,...,...,...
2094,Joiner,Zions Bancorp,ZION.OQ,2001-06-25
2095,Leaver,Zions Bancorp,ZION.OQ,2024-03-18
2096,Joiner,Zurn Industry,ZRN.N^F98,1994-12-30
2097,Leaver,Zurn Industry,ZRN.N^F98,1995-12-20


In [135]:
jl['BaseRIC'] = (
    jl['Code']
      .str.replace(r'\^.*$', '', regex=True)
)

In [136]:
jl

,Status,Issuer,Code,Date,BaseRIC
0,Joiner,Agilent,A.N,2000-06-05,A.N
1,Joiner,Alcoa Corp,AA.N,2016-11-01,AA.N
2,Leaver,Alcoa Corp,AA.N,2016-11-02,AA.N
3,Joiner,Altaba,AABA.OQ^J19,1999-12-08,AABA.OQ
4,Leaver,Altaba,AABA.OQ^J19,2017-06-19,AABA.OQ
...,...,...,...,...,...
2094,Joiner,Zions Bancorp,ZION.OQ,2001-06-25,ZION.OQ
2095,Leaver,Zions Bancorp,ZION.OQ,2024-03-18,ZION.OQ
2096,Joiner,Zurn Industry,ZRN.N^F98,1994-12-30,ZRN.N
2097,Leaver,Zurn Industry,ZRN.N^F98,1995-12-20,ZRN.N


In [137]:
records = []

for baseric, grp in jl.groupby("BaseRIC", sort=False):
    in_date = None
    issuer  = grp["Issuer"].iloc[-1]      # most-recent name
    for _, row in grp.iterrows():
        if row["Status"].startswith("Join"):
            in_date = row["Date"].date()   # open a stint
        else:  # Leaver
            records.append([
                baseric,
                row["Code"],       # <-- pull Code here
                issuer,
                in_date,
                row["Date"].date()
            ])
            in_date = None

    # Still in the index today? then End = NaT
    if in_date is not None:
        last_code = grp["Code"].iloc[-1]    # use the group’s last Code
        records.append([
            baseric,
            last_code,      # <-- and here
            issuer,
            in_date,
            pd.NaT
        ])

membership = pd.DataFrame(
    records,
    columns=["BaseRIC", "Code", "Issuer", "Start", "End"]
)


In [138]:
# 1. unique RICs
rics = membership["BaseRIC"].unique().tolist()

# 2. fetch the GICS sector for each RIC
df_sectors, err = ek.get_data(
    instruments=rics,
    fields=["TR.GICSSector"]
)

# rename for clarity
df_sectors = df_sectors.rename(
    columns={
        "Instrument": "BaseRIC",
        "TR.GICSSector": "GICS Sector"
    }
)

# Merge with the existing name
membership = membership.merge(
    df_sectors[["BaseRIC", "GICS Sector Name"]],
    on="BaseRIC",
    how="left"
)

In [139]:
membership

,BaseRIC,Code,Issuer,Start,End,GICS Sector Name
0,A.N,A.N,Agilent,2000-06-05,NaT,Health Care
1,AA.N,AA.N,Alcoa Corp,2016-11-01,2016-11-02,Materials
2,AABA.OQ,AABA.OQ^J19,Altaba,1999-12-08,2017-06-19,<NA>
3,AAL.N,AAL.N^B97,Aon Grp,1994-12-30,1997-01-14,Industrials
4,AAL.OQ,AAL.OQ,American Airline,2015-03-23,2024-09-23,Industrials
...,...,...,...,...,...,...
1273,ZE.N,ZE.N^E98,ZTAC,1994-12-30,1995-11-08,<NA>
1274,ZIMV.OQ,ZIMV.OQ,ZimVie,2022-03-01,2022-03-02,Health Care
1275,ZION.OQ,ZION.OQ,Zions Bancorp,2001-06-25,2024-03-18,Financials
1276,ZRN.N,ZRN.N^F98,Zurn Industry,1994-12-30,1995-12-20,<NA>


In [159]:
# Make sure End is a datetime
membership['End'] = pd.to_datetime(membership['End'])

# Keep only rows where End is defined and its year is between 2001 and 2024
mask = (
    membership['End'].notna() &
    membership['End'].dt.year.between(2001, 2024)
)
filtered = membership.loc[mask, ['BaseRIC', 'Code', 'Issuer', 'Start', 'End', 'GICS Sector Name']]

# Show the result
filtered

,BaseRIC,Code,Issuer,Start,End,GICS Sector Name
1,AA.N,AA.N,Alcoa Corp,2016-11-01,2016-11-02,Materials
2,AABA.OQ,AABA.OQ^J19,Altaba,1999-12-08,2017-06-19,<NA>
4,AAL.OQ,AAL.OQ,American Airline,2015-03-23,2024-09-23,Industrials
5,AAP.N,AAP.N,Advance Auto,2015-07-09,2023-08-25,Consumer Discretionary
8,ABI.N,ABI.N^K08,Applied Biosyst,1999-05-06,2008-11-24,<NA>
...,...,...,...,...,...,...
1264,XRX.OQ,XRX.OQ,Xerox Hlngs,1994-12-30,2021-03-22,Information Technology
1265,XTO.N,XTO.N^F10,XTO Energy,2004-12-29,2010-06-28,<NA>
1270,YUMC.N,YUMC.N,Yum China Hldg,2016-11-01,2016-11-02,Consumer Discretionary
1274,ZIMV.OQ,ZIMV.OQ,ZimVie,2022-03-01,2022-03-02,Health Care


In [140]:
import pandas as pd
import warnings
import eikon as ek
from datetime import datetime

# --- 0) Make sure your membership DataFrame is loaded and has Start/End columns ---

# 1) Convert Start/End to pandas Timestamps
membership["Start"] = pd.to_datetime(membership["Start"])
membership["End"]   = pd.to_datetime(membership["End"])

# 2) Identify and report any rows missing a Start date
mask_missing = membership["Start"].isna()

# 3) Fill those missing Starts with 1999-01-01
membership.loc[mask_missing, "Start"] = pd.Timestamp("1999-01-01")

In [141]:
import pandas as pd
from tqdm import tqdm
import eikon as ek
from datetime import datetime

warnings.filterwarnings("ignore", category=FutureWarning)

# 1) Define the raw field names (no inlined dates)
fields = [
    'TR.TotalEquity.periodenddate',
    'TR.TotalEquity',
    'TR.NetIncome',
    'TR.TtlLiabShareholderEqty',
    'TR.IncomeTaxRatePct',
    'TR.TotalReceivablesNet',
    'TR.F.INVNTTOT',
    'TR.F.PPENETTOT',
    'TR.Revenue',
    'TR.F.DIVYLDCOMSTOCKISSUEPCTTTM',
    'TR.F.NETINCAFTERMININTR',
    'TR.F.MKTCAP',
    'TR.F.MKTCAP3YRAVG',
    'TR.F.TOTDEBTPCTOFTOTASSETS',
    'TR.F.TOTDEBTPCTOFTOTEQ',
    'TR.F.TOTDEBTPCTOFTOTCAP',
    'TR.F.PPEEXCLROUTANGCAPLEASENETSUPPL',
    'TR.F.RND',
    'TR.F.PRICECLOSEENDPRD',
    'TR.F.PRICETOEPSDILEXCLEXORDNORMTOTTTM',
    'TR.F.EBITDA',
    'TR.F.DEBTTOT',
    'TR.TotalAssets'
]

# 2) Common static parameters
static_params = {
    'Period': 'FQ0',
    'Frq':    'FQ',
    'Scale':  6,
    'Curn':  'USD'
}

# 3) Prepare a list to collect each chunk
all_quarters = []

# 4) Loop row‐by‐row
today = datetime.today().strftime('%Y-%m-%d')
for _, row in tqdm(membership.iterrows(),
                  total=membership.shape[0],
                  desc="Fetching per‐stint fundamentals"):
    ric    = row['Code']
    sdate  = row['Start'].strftime('%Y-%m-%d')
    edate  = (row['End'].strftime('%Y-%m-%d')
              if pd.notna(row['End']) else today)
    # build the per‐row parameters
    params = {
        **static_params,
        'SDate': sdate,
        'EDate': edate
    }

    data, err = ek.get_data(
        instruments=[ric],
        fields=fields,
        parameters=params
    )
    if err:
        print(f"⚠️  Error for {ric}: {err}")
        continue

    tmp = pd.DataFrame(data)
    tmp['RIC']         = ric
    tmp['GICS Sector'] = row['GICS Sector Name']
    all_quarters.append(tmp)

# 5) Concatenate into one DataFrame
fundamentals_df = pd.concat(all_quarters, ignore_index=True)


Fetching per‐stint fundamentals:   3%|▎         | 37/1278 [00:47<26:16,  1.27s/it]

⚠️  Error for AGC.N^H01: [{'code': 416, 'col': 1, 'message': "Unable to collect data for the field 'TR.TOTALEQUITY.periodenddate' and some specific identifier(s).", 'row': 0}, {'code': 416, 'col': 2, 'message': "Unable to collect data for the field 'TR.TotalEquity' and some specific identifier(s).", 'row': 0}, {'code': 416, 'col': 3, 'message': "Unable to collect data for the field 'TR.NetIncome' and some specific identifier(s).", 'row': 0}, {'code': 416, 'col': 4, 'message': "Unable to collect data for the field 'TR.TtlLiabShareholderEqty' and some specific identifier(s).", 'row': 0}, {'code': 416, 'col': 5, 'message': "Unable to collect data for the field 'TR.IncomeTaxRatePct' and some specific identifier(s).", 'row': 0}, {'code': 416, 'col': 6, 'message': "Unable to collect data for the field 'TR.TotalReceivablesNet' and some specific identifier(s).", 'row': 0}, {'code': 416, 'col': 9, 'message': "Unable to collect data for the field 'TR.Revenue' and some specific identifier(s).", '

Fetching per‐stint fundamentals:  13%|█▎        | 170/1278 [03:26<43:18,  2.35s/it]

⚠️  Error for BOAT.O^D99: [{'code': 412, 'col': 1, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 2, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 3, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 4, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 5, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 6, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 7, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 8, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 9, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 10, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 11, 'messag

Fetching per‐stint fundamentals:  25%|██▌       | 322/1278 [06:03<14:23,  1.11it/s]

...

Fetching per‐stint fundamentals:  74%|███████▍  | 951/1278 [18:41<12:10,  2.23s/it]

⚠️  Error for PZS.N^L99: [{'code': 412, 'col': 1, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 2, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 3, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 4, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 5, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 6, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 7, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 8, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 9, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 10, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 11, 'message

Fetching per‐stint fundamentals:  97%|█████████▋| 1241/1278 [24:33<00:49,  1.33s/it]

⚠️  Error for WM.N^I08: [{'code': 416, 'col': 1, 'message': "Unable to collect data for the field 'TR.TOTALEQUITY.periodenddate' and some specific identifier(s).", 'row': 0}, {'code': 416, 'col': 2, 'message': "Unable to collect data for the field 'TR.TotalEquity' and some specific identifier(s).", 'row': 0}, {'code': 416, 'col': 3, 'message': "Unable to collect data for the field 'TR.NetIncome' and some specific identifier(s).", 'row': 0}, {'code': 416, 'col': 4, 'message': "Unable to collect data for the field 'TR.TtlLiabShareholderEqty' and some specific identifier(s).", 'row': 0}, {'code': 416, 'col': 5, 'message': "Unable to collect data for the field 'TR.IncomeTaxRatePct' and some specific identifier(s).", 'row': 0}, {'code': 416, 'col': 6, 'message': "Unable to collect data for the field 'TR.TotalReceivablesNet' and some specific identifier(s).", 'row': 0}, {'code': 416, 'col': 9, 'message': "Unable to collect data for the field 'TR.Revenue' and some specific identifier(s).", 'r

Fetching per‐stint fundamentals: 100%|██████████| 1278/1278 [25:34<00:00,  1.20s/it]


In [142]:
# How many rows total before dropping?
print("Before drop:", len(fundamentals_df))

# Drop all rows where GICS Sector == "Financials"
fundamentals_df = fundamentals_df[
    fundamentals_df["GICS Sector"] != "Financials"
].reset_index(drop=True)

# How many rows remain?
print("After drop:", len(fundamentals_df))

Before drop: 61464
After drop: 54702


In [143]:
# Pulls U.S. GDP data quarterly, and then calculates the GDP growth rate for each quarter.
USGDP = ek.get_timeseries('aUSGDPCC/CA', 
                       start_date="1999-01-01", 
                       end_date="2025-12-31", 
                       interval="quarterly")
USGDP['Return'] = USGDP['VALUE'] / USGDP['VALUE'].shift(1) - 1

In [144]:
stocksinfo_df = fundamentals_df

In [145]:
import pandas as pd
import yfinance as yf

# Download daily VIX data
vix_data = yf.download("^VIX", start="1999-01-01", end="2025-12-31")

# Resample to quarterly using the last closing value of each quarter
vix_quarterly = vix_data[['Close']].resample('Q').last()

# Reset index to ensure it's a single-level DataFrame
vix_quarterly = vix_quarterly.reset_index()

# Convert 'Date' column to quarterly periods
vix_quarterly['Period'] = vix_quarterly['Date'].dt.to_period('Q')

# FIX: Flatten MultiIndex if necessary
if isinstance(vix_quarterly.columns, pd.MultiIndex):
    vix_quarterly.columns = [col[0] for col in vix_quarterly.columns]

# Keep only 'Period' and 'Close', and rename 'Close' to 'VIX_Close'
vix_quarterly = vix_quarterly[['Period', 'Close']].rename(columns={'Close': 'VIX_Close'})

# Calculate VIX Growth Rate (quarter-over-quarter change)
vix_quarterly['VIX_Growth'] = vix_quarterly['VIX_Close'].pct_change()

# Convert 'Period End Date' in stocksinfo_df to datetime and extract quarterly period
stocksinfo_df['Period'] = pd.to_datetime(stocksinfo_df['Period End Date']).dt.to_period('Q')

# Convert Period to string in both DataFrames to ensure matching format
stocksinfo_df['Period'] = stocksinfo_df['Period'].astype(str)
vix_quarterly['Period'] = vix_quarterly['Period'].astype(str)

# Merge stocksinfo_df with vix_quarterly on 'Period'
stocksinfo_df = pd.merge(stocksinfo_df, vix_quarterly, on='Period', how='left')

[*********************100%***********************]  1 of 1 completed


In [ ]:
stocksinfo_df.to_csv('/Users/christopher/Desktop/Seminar-Advanced-Finance/stocksinfo_data.csv', index=False)

In [172]:
# Saves the dataframe
stocksinfo_df = pd.read_csv(
    '/Users/christopher/Desktop/Seminar-Advanced-Finance/stocksinfo_data_p2.csv'
)

In [173]:
# Convert 'Period End Date' in stocksinfo_df to datetime if it isn’t already
stocksinfo_df['Period End Date'] = pd.to_datetime(stocksinfo_df['Period End Date'])
# Create a new column representing the quarter period
stocksinfo_df['Period'] = stocksinfo_df['Period End Date'].dt.to_period('Q')

# Ensure USGDP's date column is datetime. If USGDP’s date is the index, you can reset the index:
if 'Date' not in USGDP.columns:
    USGDP = USGDP.reset_index()
USGDP['Date'] = pd.to_datetime(USGDP['Date'])
# Create a quarterly period column for USGDP as well
USGDP['Period'] = USGDP['Date'].dt.to_period('Q')

# Now merge on the 'Period' column
merged_df = pd.merge(stocksinfo_df, USGDP, on='Period', how='left')
stocksinfo_df = merged_df

In [175]:
# Remove rows where 'AdjustedDate' is NaT or has a year before 2000
stocksinfo_df = stocksinfo_df[
    stocksinfo_df['Period End Date'].notnull() 
    & (stocksinfo_df['Period End Date'].dt.year >= 2000) 
    & (stocksinfo_df['Period End Date'].dt.year != 2025)
]

In [ ]:
# Compute Debt Ratio:
# It measures how much of a company's assets are financed by debt.
stocksinfo_df["Debt Ratio"] = (
    (stocksinfo_df["Total Liabilities And Shareholders' Equity"] - stocksinfo_df["Total Equity"]) /
    stocksinfo_df["Total Assets"]
)

# Compute ROE:
# ROE = Net Income Incl Extra Before Distributions / Total Equity
stocksinfo_df["ROE"] = (
    stocksinfo_df["Net Income Incl Extra Before Distributions"] / stocksinfo_df["Total Equity"]
)

# Compute Asset Tangibility:
# Asset Tangibility = (Property Plant & Equipment - Net - Total + Inventories - Total) / Total Assets
stocksinfo_df["Property Plant & Equipment - Net - Total"] = stocksinfo_df["Property Plant & Equipment - Net - Total"].fillna(0)
stocksinfo_df["Inventories - Total"] = stocksinfo_df["Inventories - Total"].fillna(0)
stocksinfo_df["Asset Tangibility"] = (
    (stocksinfo_df["Property Plant & Equipment - Net - Total"] + stocksinfo_df["Inventories - Total"]) /
    stocksinfo_df["Total Assets"]
)

# Compute Tobins Q:
# Shows how highly the market values the firm's assets compared to their book value.
stocksinfo_df["TobinsQ"] = (
    (stocksinfo_df["Market Capitalization"]) /
    stocksinfo_df["Total Assets"]
)

# Compute Research and Development intensity:
# It measures how much of the company’s assets are invested in R&D activities
stocksinfo_df["RnDintensity"] = (
    (stocksinfo_df["Research & Development Expense"]) /
    stocksinfo_df["Total Assets"]
)

# Compute PE Ratio:
# Shows how much investors are willing to pay for each unit of earnings
stocksinfo_df["PEratio"] = (
    (stocksinfo_df["Market Capitalization"]) /
    stocksinfo_df["Net Income after Minority Interest"]
)

# Compute Size
# It measures firm size based on the log of total assets
# which is common in finance and economics to handle large variations between companies
stocksinfo_df['Size'] = np.log(stocksinfo_df['Total Assets'])

stocksinfo_df.rename(columns={'Dividend Yield - Common - Net - Issue - %, TTM': 'DPR'}, inplace=True)
stocksinfo_df.rename(columns={'Net Income after Minority Interest': 'Net Income'}, inplace=True)
stocksinfo_df.rename(columns={'VALUE': 'GDP'}, inplace=True)
stocksinfo_df.rename(columns={'Return': 'GDP Growth'}, inplace=True)
stocksinfo_df.rename(columns={'Earnings before Interest Taxes Depreciation & Amortization': 'EBITDA'}, inplace=True)

/var/folders/1t/g6pbwy2d0xs212cq6dztdzn80000gn/T/ipykernel_16975/4274420670.py:3:SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/1t/g6pbwy2d0xs212cq6dztdzn80000gn/T/ipykernel_16975/4274420670.py:10:SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/1t/g6pbwy2d0xs212cq6dztdzn80000gn/T/ipykernel_16975/4274420670.py:16:SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [178]:
import pandas_datareader.data as web
import datetime

# Define date range
start = datetime.datetime(1999, 1, 1)
end = datetime.datetime(2025, 12, 31)

# Get U.S. Fed Funds Rate from FRED
interest_rate_data = web.DataReader("FEDFUNDS", "fred", start, end)

# Convert index to a column
interest_rate_data.reset_index(inplace=True)

# Convert 'DATE' column to quarterly period
interest_rate_data['Period'] = interest_rate_data['DATE'].dt.to_period('Q')

# Rename columns
interest_rate_data.rename(columns={"FEDFUNDS": "Interest Rate"}, inplace=True)

# Sort by DATE in descending order to ensure the latest is first
interest_rate_data = interest_rate_data.sort_values(by="DATE", ascending=False)

# Keep only the latest entry for each Period
interest_rate_quarterly = interest_rate_data.drop_duplicates(subset=["Period"], keep="first")

# Reset index after filtering
interest_rate_quarterly = interest_rate_quarterly.reset_index(drop=True)

# Ensure both Period columns are in the same format (string)
stocksinfo_df['Period'] = stocksinfo_df['Period'].astype(str)
interest_rate_quarterly['Period'] = interest_rate_quarterly['Period'].astype(str)

# Merge stocksinfo_df with interest_rate_quarterly on 'Period'
stocksinfo_df = pd.merge(stocksinfo_df, interest_rate_quarterly, on='Period', how='left')

/var/folders/1t/g6pbwy2d0xs212cq6dztdzn80000gn/T/ipykernel_16975/2301696998.py:30:SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [179]:
import numpy as np
import statsmodels.api as sm

# Replace any +inf or -inf with NaN in your entire panel_data
final_merged_df = stocksinfo_df.replace([np.inf, -np.inf], np.nan)

In [180]:
# Create lagged debt ratio (1 period back) for each firm
final_merged_df['DebtRatio_Lag'] = final_merged_df.groupby('Instrument')['Debt Ratio'].shift(1)

# Create lagged VIX growth (1 period back) for each firm
final_merged_df['VIX_Growth_Lag'] = final_merged_df.groupby('Instrument')['VIX_Growth'].shift(1)

# Create lagged VIX growth (2 periods back) for each firm
final_merged_df['VIX_Growth_Lag2'] = final_merged_df.groupby('Instrument')['VIX_Growth'].shift(2)

# Create interaction term between debt ratio and VIX growth
final_merged_df['Debt_VIX_Interaction'] = final_merged_df['Debt Ratio'] * final_merged_df['VIX_Growth']

# Take the log(1 + Tobin's Q) to smooth values
final_merged_df['Log_TobinsQ'] = np.log1p(final_merged_df['TobinsQ'])

# Create interaction between debt ratio and firm size
final_merged_df['Debt_Size_Interaction'] = final_merged_df['Debt Ratio'] * final_merged_df['Size']

# Take the log(1 + Debt Ratio) to handle zeros and reduce skewness
final_merged_df['Log_DebtRatio'] = np.log1p(final_merged_df['Debt Ratio'])

# Interaction between logged debt ratio and size
final_merged_df['Log_Debt_Size_Interaction'] = final_merged_df['Log_DebtRatio'] * final_merged_df['Size']

# Create a squared term for debt ratio
final_merged_df['DebtRatio_sq'] = final_merged_df['Debt Ratio'] ** 2

# Fill missing R&D intensity values with 0
final_merged_df["RnDintensity"] = final_merged_df["RnDintensity"].fillna(0)

# Create a squared term for Return on Equity (ROE)
final_merged_df['ROE_sq'] = final_merged_df['ROE'] ** 2

# Sort the dataset by firm and date
final_merged_df = final_merged_df.sort_values(by=['Instrument', 'Period End Date'])

# Create lagged P/E ratio (1 period back) for each firm
final_merged_df['PEratio_lag1'] = final_merged_df.groupby('Instrument')['PEratio'].shift(1)

# Create interaction between debt ratio and lagged P/E ratio
final_merged_df['Debt_PE_lagged_interaction'] = final_merged_df['Debt Ratio'] * final_merged_df['PEratio_lag1']

# Calculate median debt ratio by industry and date
final_merged_df['Industry_Median_Leverage'] = final_merged_df.groupby(['GICS Sector', 'Period End Date'])['Debt Ratio'].transform('median')

# Take the log(1 + Net Income) to normalize the income variable
final_merged_df['Log_NetIncome'] = np.log1p(final_merged_df['Net Income'])

# Convert GDP growth to percentage form
final_merged_df['GDP_Growth_pct'] = final_merged_df['GDP Growth']*100

# Create a squared term for firm size
final_merged_df['Size_sq'] = final_merged_df['Size'] ** 2

# Create interaction between debt ratio and P/E ratio
final_merged_df['Debt_PE_interaction'] = final_merged_df['Debt Ratio'] * final_merged_df['PEratio']

# Fill DPR with 0 if it is blank
final_merged_df['DPR'] = final_merged_df['DPR'].fillna(0)

/Users/christopher/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:399:RuntimeWarning: divide by zero encountered in log1p
/Users/christopher/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:399:RuntimeWarning: invalid value encountered in log1p


In [181]:
# Create a constant to shift Net Income so all values become positive
constant_shift = abs(final_merged_df['Net Income'].min()) + 1

# Create a new variable by taking the natural log of the shifted Net Income
final_merged_df['Log_NetIncome_Shifted'] = np.log(final_merged_df['Net Income'] + constant_shift)

In [182]:
# Create a constant to shift EBITDA so all values become positive
constant_shift = abs(final_merged_df['EBITDA'].min()) + 1

# Create a new variable by taking the natural log of the shifted EBITDA
final_merged_df['Log_EBITDA'] = np.log(final_merged_df['EBITDA'] + constant_shift)

# Create interaction between debt ratio and logged EBITDA
final_merged_df['Debt_LogEBITDA_Interaction'] = final_merged_df['Debt Ratio'] * final_merged_df['Log_EBITDA']

# Create interaction between debt ratio and raw (non-logged) EBITDA
final_merged_df['Debt_EBITDA_Interaction'] = final_merged_df['Debt Ratio'] * final_merged_df['EBITDA']

In [183]:
panel_data = final_merged_df.set_index(['Instrument', 'Period End Date'])
xx = panel_data[['Log_TobinsQ','Debt Ratio' , 'ROE', 'Asset Tangibility', 'Size', 'DPR', 'Log_NetIncome_Shifted', 'GDP_Growth_pct', 'VIX_Growth', 'RnDintensity', 'PEratio', 'Interest Rate', 'EBITDA', 'Industry_Median_Leverage']]

In [184]:
# xx is the dataframe
summary_stats = xx.describe().T  # Transpose to match LaTeX table format

# Round numbers for better readability
summary_stats = summary_stats[['mean', 'std', 'min', 'max']].round(3)

# Rename standard deviation column
summary_stats.rename(columns={'std': 'sd'}, inplace=True)

# Add the observation row properly
summary_stats.loc["Observations"] = [f"{xx.shape[0]:,}", "", "", ""]

# Convert to LaTeX with proper formatting
latex_code = summary_stats.to_latex(
    column_format="lcccc",  # Ensures alignment
    escape=False,
    caption="Deskriptiv statistik",
    label="tab:summary"
)

# Ensure Observations row is well-placed
latex_code = latex_code.replace(
    r"\bottomrule",
    f"Observations & {xx.shape[0]:,} & & & \\\\\n\ bottomrule"
)

# Save to file
with open("summary_statistics.tex", "w") as f:
    f.write(latex_code)

print(latex_code)  # Print LaTeX table


\begin{table}
\caption{Deskriptiv statistik}
\label{tab:summary}
\begin{tabular}{lcccc}
\toprule
 & mean & sd & min & max \\
\midrule
Log_TobinsQ & 0.854866 & 0.504118 & 0.000006 & 4.968283 \\
Debt Ratio & 0.616017 & 0.225108 & 0.031676 & 3.621794 \\
ROE & 0.034649 & 1.484558 & -104.611111 & 140.339244 \\
Asset Tangibility & 0.397537 & 5.434533 & -24.315227 & 839.406727 \\
Size & 9.525145 & 1.204748 & 5.784988 & 13.908261 \\
DPR & 1.358183 & 3.525931 & -7.515152 & 339.508171 \\
Log_NetIncome_Shifted & 10.689986 & 0.062529 & 0.000000 & 11.288068 \\
GDP_Growth_pct & 0.343993 & 1.253473 & -7.970840 & 7.775070 \\
VIX_Growth & 0.054526 & 0.432840 & -0.460661 & 2.885341 \\
RnDintensity & 0.005618 & 0.011918 & -0.073164 & 0.394303 \\
PEratio & -0.059482 & 13.467237 & -2750.290360 & 1.356954 \\
Interest Rate & 1.925938 & 2.018782 & 0.070000 & 6.530000 \\
EBITDA & 943.399603 & 2177.466650 & -89284.000000 & 45912.000000 \\
Industry_Median_Leverage & 0.611476 & 0.121916 & 0.093862 & 3.621794 \\
O

In [185]:
final_merged_df.to_csv('/Users/christopher/Desktop/Seminar-Advanced-Finance/finaldata_p2.csv', index=False)

In [186]:
unique_n = final_merged_df["Instrument"].nunique()   # or "RIC" / "Ticker" / whatever the column is called
print(f"There are {unique_n:,} unique instruments.")

There are 921 unique instruments.
